In [8]:
import itertools

import numpy as np
import pandas as pd

from aif360.metrics import BinaryLabelDatasetMetric 
from aif360.metrics.mdss_classification_metric import MDSSClassificationMetric
from aif360.detectors.mdss.ScoringFunctions.Bernoulli import Bernoulli

In [9]:
df = pd.read_csv('Taipei City Government Transportation Bureau 2020 Traffic Accident Dataset bias.csv')
df.head()

,Gender,Delivery_Type,Age,Vehicle types,Cause code-main,Vehicle purpose,Purpose of trip,Weather,Light,Speed limit,Signal 1,Signal 2,Day and night,Degree of injury,Accident type
0,3,1,0,G06,#,#,#,#,#,50,#,#,Night,0,#
1,1,0,55,H03,#,#,#,#,#,50,#,#,Night,0,#
2,1,0,42,B12,#,#,#,#,#,40,#,#,Night,0,#
3,1,0,40,B01,#,#,#,#,#,40,#,#,Night,0,#
4,3,0,0,#,#,#,#,#,#,#,#,#,Night,0,#


In [49]:
dfcutQ = df[['Gender','Delivery_Type', 'Age', 'Vehicle types','Accident type','Degree of injury']]
dfcutQ

,Gender,Delivery_Type,Age,Vehicle types,Accident type,Degree of injury
0,3,1,0,G06,#,0
1,1,0,55,H03,#,0
2,1,0,42,B12,#,0
3,1,0,40,B01,#,0
4,3,0,0,#,#,0
...,...,...,...,...,...,...
213866,3,0,0,B03,16,3
213867,1,0,54,B12,17,1
213868,1,0,42,B01,#,0
213869,1,0,19,C03,17,1


In [50]:
ix = df['Delivery_Type'] != '#'
ix = (df['Age'] != '#') & ix
ix = (df['Gender'] != '#') & ix 
ix = (df['Gender'] != '3') & ix
ix = (df['Gender'] != '4') & ix
ix = (df['Vehicle types'] !='#' ) & ix
dfcutQ = dfcutQ.loc[ix,:]
dfcutQ

,Gender,Delivery_Type,Age,Vehicle types,Accident type,Degree of injury
1,1,0,55,H03,#,0
2,1,0,42,B12,#,0
3,1,0,40,B01,#,0
5,1,0,40,B03,#,0
6,1,0,20,B03,#,0
...,...,...,...,...,...,...
213865,2,0,22,H02,16,3
213867,1,0,54,B12,17,1
213868,1,0,42,B01,#,0
213869,1,0,19,C03,17,1


In [51]:
def quantizeCarCategory(x):
    if x == 'B03':
        return 1.0
    elif x == 'C03':
        return 2.0
    else:
        return 0.0
    
def quantizeSex(x):
    if x == '2':
        return 0.0
    else:
        return 1.0

    
def quantizeDelivery_Type(x):
    if x != '0':
        return 1.0
    else:
        return 0.0

def quantizeAge(x):
    if int(x) <= 18:
        return 1.0
    elif int(x) >= 65:
        return 3.0
    else:
        return 2.0
    
def quantizeinjury(x):
    if x == 0:
        return 1.0
    else:
        return 0.0
    
def quantizeinaccident(x):
    if x == '#':
        return 0.0
    else:
        return 1.0

In [52]:
dfcutQ['Vehicle types'] = dfcutQ['Vehicle types'].apply(lambda x: quantizeCarCategory(x))
dfcutQ['Delivery_Type'] = dfcutQ['Delivery_Type'].apply(lambda x: quantizeDelivery_Type(x))
dfcutQ['Age'] = dfcutQ['Age'].apply(lambda x: quantizeAge(x))
dfcutQ['Gender'] = dfcutQ['Gender'].apply(lambda x: quantizeSex(x))
dfcutQ['Accident type'] = dfcutQ['Accident type'].apply(lambda x: quantizeinaccident(x))
dfcutQ['Degree of injury'] = dfcutQ['Degree of injury'].apply(lambda x: quantizeinjury(x))

dfcutQ=dfcutQ.rename(columns={'Degree of injury':'Is there no injury?'})
dfcutQ.tail()


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

,Gender,Delivery_Type,Age,Vehicle types,Accident type,Is there no injury?
213865,0.0,0.0,2.0,0.0,1.0,0.0
213867,1.0,0.0,2.0,0.0,1.0,0.0
213868,1.0,0.0,2.0,0.0,0.0,1.0
213869,1.0,0.0,2.0,2.0,1.0,0.0
213870,1.0,0.0,2.0,0.0,0.0,1.0


In [56]:
from sklearn.linear_model import LogisticRegression
X = dfcutQ.drop('Is there no injury?', axis = 1)
y = dfcutQ['Is there no injury?']
clf = LogisticRegression(solver='lbfgs', C=1.0, penalty='l2')
clf.fit(X, y)

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.


LogisticRegression()

In [57]:
probs = pd.Series(clf.predict_proba(X)[:,0])

is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.


In [58]:
from aif360.detectors.mdss_detector import bias_scan
privileged_subset = bias_scan(data=X,observations=y,expectations=probs,favorable_value=0, overpredicted=True)
unprivileged_subset = bias_scan(data=X,observations=y,expectations=probs,favorable_value=0,overpredicted=False)
print(privileged_subset)
print(unprivileged_subset)

({'Accident type': [0.0]}, 14.3024)
({'Delivery_Type': [0.0], 'Gender': [0.0, 1.0], 'Accident type': [1.0], 'Vehicle types': [0.0, 2.0], 'Age': [2.0]}, 4.8618)


In [59]:
dff = X.copy()
dff['observed'] = y 
dff['probabilities'] = 1 - probs

In [60]:
to_choose = dff[privileged_subset[0].keys()].isin(privileged_subset[0]).all(axis=1)
temp_df = dff.loc[to_choose]

In [61]:
"Our detected priviledged group has a size of {}, we observe {} as the average Degree of injury, but our model predicts {}"\
.format(len(temp_df), temp_df['observed'].mean(), temp_df['probabilities'].mean())

'Our detected priviledged group has a size of 142966, we observe 1.0 as the average Degree of injury, but our model predicts 0.7166889799828895'

In [62]:
to_choose = dff[unprivileged_subset[0].keys()].isin(unprivileged_subset[0]).all(axis=1)
temp_df = dff.loc[to_choose]

In [63]:
"Our detected priviledged group has a size of {}, we observe {} as the average Degree of injury, but our model predicts {}"\
.format(len(temp_df), temp_df['observed'].mean(), temp_df['probabilities'].mean())

'Our detected priviledged group has a size of 37962, we observe 0.004372793846478057 as the average Degree of injury, but our model predicts 0.7152690621097503'